In [1]:
import pandas as pd

# Load the uploaded market data CSV
file_path = "market_data.csv"
df = pd.read_csv(file_path)

# Display the first few rows and info for inspection
df.head(), df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126117 entries, 0 to 126116
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   126117 non-null  int64  
 1   currentdate  126117 non-null  object 
 2   currenttime  126117 non-null  object 
 3   high         126117 non-null  float64
 4   low          126117 non-null  float64
 5   close        126117 non-null  float64
 6   symbol       126117 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 6.7+ MB


(   Unnamed: 0 currentdate currenttime    high     low   close    symbol
 0           0    1/1/2024     9:15:00  654.35  648.35  654.35  AARTIIND
 1           1    1/1/2024     9:20:00  654.75  652.10  653.00  AARTIIND
 2           2    1/1/2024     9:25:00  653.35  651.00  652.80  AARTIIND
 3           3    1/1/2024     9:30:00  652.95  651.10  652.45  AARTIIND
 4           4    1/1/2024     9:35:00  654.00  652.50  653.75  AARTIIND,
 None)

In [7]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv("market_data.csv")  # Replace with your file path if needed

# Parse dates correctly (format is MM/DD/YYYY)
df['currentdate'] = pd.to_datetime(df['currentdate'], format='%m/%d/%Y')
df['datetime'] = pd.to_datetime(df['currentdate'].astype(str) + ' ' + df['currenttime'])
df.sort_values(by=['symbol', 'datetime'], inplace=True)

# ORB strategy settings
TARGET_PCT = 0.005    # +0.5%
STOP_LOSS_PCT = 0.0025  # -0.25%
EXIT_TIME = "15:15:00"

# Store trades
trade_logs = []

# Group by symbol and date
for (symbol, date), group in df.groupby(['symbol', 'currentdate']):
    group = group.sort_values(by='datetime').reset_index(drop=True)

    first_candle = group[group['currenttime'] == '9:15:00']
    if first_candle.empty:
        continue

    high_open = first_candle.iloc[0]['high']
    low_open = first_candle.iloc[0]['low']

    buy_made = False
    sell_made = False

    for i, row in group.iterrows():
        price = row['close']
        dt = row['datetime']
        time = row['currenttime']

        # Buy logic
        if not buy_made and price > high_open:
            entry_price = price
            target_price = entry_price * (1 + TARGET_PCT)
            sl_price = entry_price * (1 - STOP_LOSS_PCT)

            for j in range(i + 1, len(group)):
                next_row = group.iloc[j]
                next_price = next_row['close']
                next_dt = next_row['datetime']
                next_time = next_row['currenttime']

                if next_price >= target_price:
                    trade_logs.append([date, symbol, "Buy", dt.time(), entry_price, next_dt.time(), next_price, next_price - entry_price, "Target Hit"])
                    break
                elif next_price <= sl_price:
                    trade_logs.append([date, symbol, "Buy", dt.time(), entry_price, next_dt.time(), next_price, next_price - entry_price, "Stop Loss Hit"])
                    break
                elif next_time == EXIT_TIME:
                    trade_logs.append([date, symbol, "Buy", dt.time(), entry_price, next_dt.time(), next_price, next_price - entry_price, "Time Exit"])
                    break
            buy_made = True

        # Sell logic
        if not sell_made and price < low_open:
            entry_price = price
            target_price = entry_price * (1 - TARGET_PCT)
            sl_price = entry_price * (1 + STOP_LOSS_PCT)

            for j in range(i + 1, len(group)):
                next_row = group.iloc[j]
                next_price = next_row['close']
                next_dt = next_row['datetime']
                next_time = next_row['currenttime']

                if next_price <= target_price:
                    trade_logs.append([date, symbol, "Sell", dt.time(), entry_price, next_dt.time(), next_price, entry_price - next_price, "Target Hit"])
                    break
                elif next_price >= sl_price:
                    trade_logs.append([date, symbol, "Sell", dt.time(), entry_price, next_dt.time(), next_price, entry_price - next_price, "Stop Loss Hit"])
                    break
                elif next_time == EXIT_TIME:
                    trade_logs.append([date, symbol, "Sell", dt.time(), entry_price, next_dt.time(), next_price, entry_price - next_price, "Time Exit"])
                    break
            sell_made = True

# Create DataFrame for results
columns = ["Date", "Symbol", "TradeType", "EntryTime", "EntryPrice", "ExitTime", "ExitPrice", "PnL", "ExitReason"]
trades_df = pd.DataFrame(trade_logs, columns=columns)

# ✅ Display the first few trades
print("ORB Strategy Trade Log:\n")
print(trades_df.head(10))  # Show first 10 trades


ORB Strategy Trade Log:

        Date    Symbol TradeType EntryTime  EntryPrice  ExitTime  ExitPrice  \
0 2024-01-01  AARTIIND       Buy  09:45:00      655.05  09:55:00     653.00   
1 2024-01-02  AARTIIND       Buy  09:30:00      658.10  09:50:00     655.55   
2 2024-01-02  AARTIIND      Sell  09:55:00      651.70  10:25:00     646.80   
3 2024-01-03  AARTIIND      Sell  09:35:00      647.90  09:50:00     649.75   
4 2024-01-04  AARTIIND      Sell  09:25:00      649.85  09:40:00     652.30   
5 2024-01-05  AARTIIND      Sell  09:25:00      644.95  09:40:00     646.85   
6 2024-01-08  AARTIIND      Sell  09:25:00      611.80  09:30:00     608.50   
7 2024-01-09  AARTIIND      Sell  09:40:00      599.90  09:45:00     601.75   
8 2024-01-10  AARTIIND      Sell  09:20:00      601.60  09:30:00     603.50   
9 2024-01-11  AARTIIND       Buy  09:40:00      612.25  10:10:00     615.80   

    PnL     ExitReason  
0 -2.05  Stop Loss Hit  
1 -2.55  Stop Loss Hit  
2  4.90     Target Hit  
3 -1.

In [8]:
trades_df.to_csv("orb_trade_log.csv", index=False)
